In [1]:
import openai
import requests
from urllib import parse
import os
import dotenv

dotenv.load_dotenv()

openai.api_key = os.environ.get("OPENAI_API_KEY")
username = os.environ.get("GITHUB_USER")
password = os.environ.get("GITHUB_PASSWORD")
org = 'Significant-Gravitas'
repo = 'Auto-GPT'
branch = 'agent-state-encapsulation'
limit = 10  # Specify the number of commits to fetch

encoded_branch = parse.quote(branch)
API_URL = f"https://api.github.com/repos/{org}/{repo}/commits?sha={encoded_branch}&per_page={limit}"
print(API_URL)


def get_recent_commits(api_url, username, password):
    """Fetch the recent commits from the GitHub API."""
    response = requests.get(api_url, auth=(username, password))
    response.raise_for_status()  # Raise an exception if the request failed
    return response.json()


def get_commit_diff(commit_url, username, password):
    """Fetch the diff of a commit from the GitHub API."""
    response = requests.get(commit_url, auth=(username, password))
    response.raise_for_status()  # Raise an exception if the request failed
    return response.json()["files"]


def generate_commit_summary(commit_data):
    """Generate a descriptive summary for a commit using the OpenAI API."""
    message = commit_data["commit"]["message"]
    author = commit_data["commit"]["author"]["name"]
    date = commit_data["commit"]["author"]["date"]
    diff = get_commit_diff(commit_data["url"], username, password)

    # Define the conversation messages
    messages = [
        {"role": "system", "content": "You are a Senior Python Developer that reviews PRs and posts brief .md ."},
        {"role": "user",
         "content": f"On {date}, {author} made a commit with the following message: {message}. The diff for this commit is: {diff}. Can you summarize this commit?"}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages
        )
        return response.choices[0].message['content']
    except openai.InvalidRequestError as e:
        print(f"Failed to generate summary for commit {commit_data['sha']}: {e}")
        return None


def main():
    # Fetch the recent commits
    commits = get_recent_commits(API_URL, username, password)

    # Open the markdown file in append mode
    with open(f'commit_summaries_{org}_{repo}_{branch}.md', 'a') as f:
        # Generate and write a summary for each commit
        for commit in commits:
            summary = generate_commit_summary(commit)
            if summary is not None:  # Only write summaries that were successfully generated
                f.write(summary)
                f.write("\n---\n")


if __name__ == "__main__":
    main()


https://api.github.com/repos/Significant-Gravitas/Auto-GPT/commits?sha=agent-state-encapsulation&per_page=10
Failed to generate summary for commit a45850d417c7f76be9d0f595a36efa9008403e4c: This model's maximum context length is 8192 tokens. However, your messages resulted in 116387 tokens. Please reduce the length of the messages.
Failed to generate summary for commit 6c4426d8e654930c8c3e25fb30565c9cc768ed3d: This model's maximum context length is 8192 tokens. However, your messages resulted in 35871 tokens. Please reduce the length of the messages.


In [2]:
import openai

def load_commit_summaries(filename):
    with open(filename, "r") as file:
        return file.read()

def generate_summary(content):
    openai.api_key = os.environ.get("OPENAI_API_KEY")

    messages = [
        {"role": "system", "content": "Your goal is to write a detailed blog post of the most recent changes to the Github Repository."},
        {"role": "user", "content": f"Your goal is to make a summary blog post about the following commit changes: {content}. Organize the blogpost structure in sections like Features, CI/CD, Testing, and so on. Remember to make an intro and conclusion for the readers including the time and contributors."}
    ]


    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            max_tokens=3000
        )
        return response.choices[0].message['content']
    except openai.InvalidRequestError as e:
        print(f"Failed to generate summary: {e}")
        return None

def main():
    commit_summaries = load_commit_summaries(f"commit_summaries_{org}_{repo}_{branch}.md")
    summary = generate_summary(commit_summaries)
    with open(f'summary_{org}_{repo}_{branch}.md', 'w') as f:
        f.write(summary)
        
if __name__ == "__main__":
    main()


In [3]:
# PR Summaries
def get_recent_prs(api_url, username, password):
    """Fetch the recent pull requests from the GitHub API."""
    response = requests.get(api_url, auth=(username, password))
    response.raise_for_status()  # Raise an exception if the request failed
    return response.json()


def generate_pr_summary(pr_data):
    """Generate a descriptive summary for a pull request using the OpenAI API."""
    title = pr_data["title"]
    user = pr_data["user"]["login"]
    created_at = pr_data["created_at"]
    html_url = pr_data["html_url"]

    # Define the conversation messages
    messages = [
        {"role": "system", "content": "You are a Senior Python Developer reviewing PRs."},
        {"role": "user",
         "content": f"A pull request titled '{title}' was created by {user} on {created_at}. You can find it [here]({html_url}). Can you provide a summary of this PR? Use this structure: [date, pr number, Authors, size, changes] "}
    ]

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages
        )
        return response.choices[0].message['content']
    except openai.InvalidRequestError as e:
        print(f"Failed to generate summary for PR {pr_data['number']}: {e}")
        return None


def main():
    # Fetch the recent pull requests
    prs = get_recent_prs(API_URL, username, password)

    # Open the markdown file in append mode
    with open(f'pr_summaries_{org}_{repo}_{branch}.md', 'a') as f:
        # Generate and write a summary for each pull request
        for pr in prs:
            summary = generate_pr_summary(pr)
            if summary is not None:  # Only write summaries that were successfully generated
                f.write(summary)
                f.write("\n---\n")


if __name__ == "__main__":
    main()


https://api.github.com/repos/Significant-Gravitas/Auto-GPT/pulls?state=open&sort=created&direction=desc&per_page=10


In [4]:
import openai

def load_commit_summaries(filename):
    with open(filename, "r") as file:
        return file.read()

def generate_summary(content):
    openai.api_key = os.environ.get("OPENAI_API_KEY")

    messages = [
        {"role": "system", "content": "Your goal is to write a detailed blog post of the most recent pull requests to the Github Repository."},
        {"role": "user", "content": f"Your goal is to make a summary blog post about the following PRs: {content}. Organize the blogpost structure in sections like Features, CI/CD, Testing, and so on. Remember to make an intro and conclusion for the readers including the time and contributors."}
    ]


    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages,
            max_tokens=3000
        )
        return response.choices[0].message['content']
    except openai.InvalidRequestError as e:
        print(f"Failed to generate summary: {e}")
        return None

def main():
    commit_summaries = load_commit_summaries(f"commit_summaries_{org}_{repo}_{branch}.md")
    summary = generate_summary(commit_summaries)
    with open(f'pr_summaries_{org}_{repo}_{branch}.md', 'w') as f:
        f.write(summary)
        
if __name__ == "__main__":
    main()
